# Week 3 - Sensitivity analysis

This exercise uses the same predator-prey model we used for the multi-model exercise, focusing on the Python version. As with the other exercise, define a model object for the function below, with the uncertainty ranges provided:

|Parameter	|Range or value	        |
|-----------|--------------:|
|prey_birth_rate    	|0.015 – 0.035	|
|predation_rate|0.0005 – 0.003 	|
|predator_efficiency     	|0.001 – 0.004	    |
|predator_loss_rate	    |0.04 – 0.08	    |

In [109]:
import numpy as np
import matplotlib.pyplot as plt

from ema_workbench import (Model, RealParameter, TimeSeriesOutcome, perform_experiments, ema_logging)

from ema_workbench import Samplers

from ema_workbench.analysis import feature_scoring
from ema_workbench.analysis.scenario_discovery_util import RuleInductionType
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from SALib.analyze import sobol

In [53]:
def pred_prey(prey_birth_rate=0.025, predation_rate=0.0015, predator_efficiency=0.002,
             predator_loss_rate=0.06, initial_prey=50, initial_predators=20, dt=0.25,
             final_time=365, reps=1):

    #Initial values
    predators = np.zeros((reps, int(final_time/dt)+1))
    prey = np.zeros((reps, int(final_time/dt)+1))
    sim_time = np.zeros((reps, int(final_time/dt)+1))

    for r in range(reps):

        predators[r,0] = initial_predators
        prey[r,0] = initial_prey

    #Calculate the time series
    for t in range(0, sim_time.shape[1]-1):

        dx = (prey_birth_rate*prey[r,t]) - (predation_rate*prey[r,t]*predators[r,t])
        dy = (predator_efficiency*predators[r,t]*prey[r,t]) - (predator_loss_rate*predators[r,t])

        prey[r,t+1] = max(prey[r,t] + dx*dt, 0)
        predators[r,t+1] = max(predators[r,t] + dy*dt, 0)
        sim_time[r,t+1] = (t+1)*dt

    #Return outcomes
    return {'TIME':sim_time,
            'predators':predators,
            'prey':prey}

#### 1. Sensitivity analysis
Sensitivity analysis often focuses on the final values of an outcome at the end of the simulation. However, we can also look at metrics that give us additional information about the behavior of the model over time. Using [the statsmodel library](https://www.statsmodels.org/stable/index.html) and an appropriate sampling design, fit a linear regression model for each of the following indicators. What can we conclude about the behavior of the model, and about the importance of the different inputs?

  * The final values of the _prey_ outcome
  * The mean values of the _prey_ outcome over time, within each experiment
  * The standard deviations of the _prey_ outcome over time, within each experiment

In [54]:
model = Model('predprey', function=pred_prey)

model.uncertainties = [RealParameter('prey_birth_rate', 0.015, 0.035),
                 RealParameter('predation_rate', 0.0005, 0.003),
                 RealParameter('predator_efficiency', 0.001, 0.004),
                 RealParameter('predator_loss_rate', 0.04, 0.08)] 

model.outcomes = [TimeSeriesOutcome('TIME', function=np.squeeze),
            TimeSeriesOutcome('predators', function=np.squeeze),
            TimeSeriesOutcome('prey', function=np.squeeze)]



In [55]:
experiments, outcomes = perform_experiments(model, 50)

100%|█████████████████████████████████████████| 50/50 [00:00<00:00, 186.88it/s]


In [74]:
experiments

,predation_rate,predator_efficiency,predator_loss_rate,prey_birth_rate,scenario,policy,model
0,0.002584,0.003410,0.047165,0.023696,100,None,predprey
1,0.001636,0.003548,0.059306,0.022719,101,None,predprey
2,0.002199,0.003777,0.078336,0.020733,102,None,predprey
3,0.002796,0.002658,0.058578,0.034031,103,None,predprey
4,0.001715,0.003910,0.046317,0.031937,104,None,predprey
5,0.002385,0.001986,0.060560,0.021027,105,None,predprey
6,0.001975,0.002333,0.071499,0.030679,106,None,predprey
7,0.001216,0.001121,0.047791,0.024172,107,None,predprey
8,0.000620,0.001775,0.076598,0.033531,108,None,predprey
9,0.002433,0.002789,0.067440,0.029236,109,None,predprey


In [96]:
import statsmodels.api as sm
prey_list = outcomes["prey"]
prey_list_stdiv = []
for scenario in prey_list:
    prey_list_stdiv.append(np.std(scenario))
   
y = prey_list_stdiv
predation_values = experiments["predation_rate"]
pred_eff = experiments ["predator_efficiency"]
pred_rate = experiments["predation_rate"]
prey_birth_rate = experiments ["prey_birth_rate"]

X = [predation_values, pred_eff, pred_rate, prey_birth_rate]

[15.566654957348701,
 14.56656429237974,
 14.211676269310926,
 16.195992573108782,
 14.422214664835087,
 15.59487371055699,
 12.462742558843754,
 5.022348342500627,
 19.215742514054952,
 15.568806462757195,
 15.878073920882535,
 3.235599090783013,
 13.784840870589246,
 14.955702613325196,
 15.793702252324511,
 16.24966073821136,
 13.903861450843207,
 12.86379443574068,
 15.0498338588872,
 17.392175403222193,
 18.45713461757396,
 5.381891262811818,
 15.90454839372382,
 16.46939697264899,
 16.307059689699155,
 10.404140759832638,
 14.890949415161092,
 14.024545510854725,
 15.205646013115002,
 4.627168426956481,
 16.22496880918244,
 11.782764304053572,
 14.099275967531447,
 15.618060425806442,
 16.4562258890409,
 15.465854837357167,
 14.862851632687722,
 1.7697212080875246,
 14.023763930037608,
 14.649229555506652,
 15.53303794655509,
 14.858296855047017,
 18.159008713439903,
 13.955777357225978,
 16.112268605499644,
 16.433732241769427,
 15.936998110073958,
 14.802055181880146,
 15.42258

In [97]:
for x in X: 
    results = sm.OLS(y, x).fit()
    print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.841
Model:                            OLS   Adj. R-squared (uncentered):              0.838
Method:                 Least Squares   F-statistic:                              259.1
Date:                Mon, 08 May 2023   Prob (F-statistic):                    3.37e-21
Time:                        15:04:34   Log-Likelihood:                         -158.80
No. Observations:                  50   AIC:                                      319.6
Df Residuals:                      49   BIC:                                      321.5
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------

#### 2. SOBOL
Use the Sobol sampling functionality included in the Workbench to perform experiments with a sample size of N=50, then analyze the results with SALib for the same three indicators. This requires specifying the keyword argument `'uncertainty_sampling'` of perform_experiments. Note that when using Sobol sampling, the meaning of the keyword argument `scenarios` changes a bit. In order to properly estimate Sobol scores as well as interaction effects, you require N * (2D+2) scenarios, where D is the number of uncertain parameters, and N is the value for scenarios passed to `perform_experiments`. Repeat the analysis for larger sample sizes, with N=250 and N=1000. How can we interpret the first-order and total indices? Are these sample sizes sufficient for a stable estimation of the indices? You'll need to use the [get_SALib_problem](https://emaworkbench.readthedocs.io/en/latest/ema_documentation/em_framework/salib_samplers.html) function to convert your Workbench experiments to a problem definition that you can pass to the SALib analysis function.

*hint*: sobol is a deterministic sequence of quasi random numbers. Thus, you can run with N=1000 and simply use slicing to get the results for N=50 and N=250.

In [100]:
scenarios = 1000*(2*4+2)
experiments, outcomes = perform_experiments(model, scenarios, uncertainty_sampling = Samplers.SOBOL)

C:\Users\rvrij\anaconda\envs\Model_based_decisions\Lib\site-packages\scipy\stats\_qmc.py:763: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  sample = self._random(n, workers=workers)
100%|█████████████████████████████████| 100000/100000 [09:04<00:00, 183.64it/s]


In [130]:
print(outcomes['prey'].shape)

(100000, 1461)


In [139]:
y = outcomes["prey"][:, -1]

In [146]:
problem = get_SALib_problem(model.uncertainties)
Si = sobol.analyze(problem, y, calc_second_order=True, print_to_console=True)

                           ST   ST_conf
predation_rate       0.133715  0.008247
predator_efficiency  0.816481  0.024410
predator_loss_rate   0.616030  0.019290
prey_birth_rate      0.544365  0.018320
                           S1   S1_conf
predation_rate      -0.004606  0.010209
predator_efficiency  0.350060  0.022876
predator_loss_rate   0.063256  0.020517
prey_birth_rate     -0.005868  0.020369
                                                 S2   S2_conf
(predation_rate, predator_efficiency)      0.004981  0.018157
(predation_rate, predator_loss_rate)       0.018799  0.016760
(predation_rate, prey_birth_rate)          0.009777  0.014717
(predator_efficiency, predator_loss_rate)  0.019036  0.034335
(predator_efficiency, prey_birth_rate)     0.009573  0.032341
(predator_loss_rate, prey_birth_rate)      0.080180  0.030595


{'S1': array([-0.00460575,  0.35006035,  0.06325598, -0.0058679 ]),
 'S1_conf': array([0.00959525, 0.02546204, 0.02191517, 0.02027245]),
 'ST': array([0.13371464, 0.81648096, 0.6160304 , 0.54436491]),
 'ST_conf': array([0.00902261, 0.02438088, 0.01999507, 0.01945364]),
 'S2': array([[       nan, 0.00498096, 0.0187995 , 0.00977713],
        [       nan,        nan, 0.01903603, 0.00957253],
        [       nan,        nan,        nan, 0.08017963],
        [       nan,        nan,        nan,        nan]]),
 'S2_conf': array([[       nan, 0.01866733, 0.01477116, 0.01363538],
        [       nan,        nan, 0.03668662, 0.03281142],
        [       nan,        nan,        nan, 0.02869594],
        [       nan,        nan,        nan,        nan]])}

{'S1': array([-0.00460575,  0.35006035,  0.06325598, -0.0058679 ]),
 'S1_conf': array([0.00959525, 0.02546204, 0.02191517, 0.02027245]),
 'ST': array([0.13371464, 0.81648096, 0.6160304 , 0.54436491]),
 'ST_conf': array([0.00902261, 0.02438088, 0.01999507, 0.01945364]),
 'S2': array([[       nan, 0.00498096, 0.0187995 , 0.00977713],
        [       nan,        nan, 0.01903603, 0.00957253],
        [       nan,        nan,        nan, 0.08017963],
        [       nan,        nan,        nan,        nan]]),
 'S2_conf': array([[       nan, 0.01866733, 0.01477116, 0.01363538],
        [       nan,        nan, 0.03668662, 0.03281142],
        [       nan,        nan,        nan, 0.02869594],
        [       nan,        nan,        nan,        nan]])}

#### 3. Extra trees
Use the [Extra-Trees analysis](https://emaworkbench.readthedocs.io/en/latest/ema_documentation/analysis/feature_scoring.html) included in the Workbench to approximate the Sobol total indices, with a suitable sampling design. As a starting point, use an ensemble of 100 trees and a max_features parameter of 0.6, and set the analysis to regression mode. Are the estimated importances stable relative to the sample size and the analysis parameters? How do the results compare to the Sobol indices? For more details on this analysis see [Jaxa-Rozen & Kwakkel (2018)](https://www.sciencedirect.com/science/article/pii/S1364815217311581)